In [3]:
import cv2
import numpy as np

In [4]:
import cv2
import numpy as np
from transformers import TrOCRProcessor, VisionEncoderDecoderModel
from PIL import Image
import torch
from transformers.utils import logging
logging.set_verbosity_info()


In [5]:
# Load model & processor
#use base instead of small if bad accuracy irl
processor = TrOCRProcessor.from_pretrained('microsoft/trocr-small-handwritten',use_fast=False)
model = VisionEncoderDecoderModel.from_pretrained('microsoft/trocr-small-handwritten')


loading configuration file preprocessor_config.json from cache at C:\Users\mahmoud\.cache\huggingface\hub\models--microsoft--trocr-small-handwritten\snapshots\b4648cfa171985a6745f37ddd637e98c0da958ac\preprocessor_config.json
size should be a dictionary on of the following set of keys: ({'width', 'height'}, {'shortest_edge'}, {'shortest_edge', 'longest_edge'}, {'longest_edge'}, {'max_width', 'max_height'}), got 384. Converted to {'height': 384, 'width': 384}.
crop_size should be a dictionary on of the following set of keys: ({'width', 'height'}, {'shortest_edge'}, {'shortest_edge', 'longest_edge'}, {'longest_edge'}, {'max_width', 'max_height'}), got 224. Converted to {'height': 224, 'width': 224}.
Image processor DeiTImageProcessor {
  "crop_size": {
    "height": 224,
    "width": 224
  },
  "do_center_crop": false,
  "do_normalize": true,
  "do_rescale": true,
  "do_resize": true,
  "image_mean": [
    0.5,
    0.5,
    0.5
  ],
  "image_processor_type": "DeiTImageProcessor",
  "image

config.json:   0%|          | 0.00/4.21k [00:00<?, ?B/s]

C:\Users\mahmoud\AppData\Local\anaconda3\Lib\site-packages\huggingface_hub\file_download.py:144: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\mahmoud\.cache\huggingface\hub\models--microsoft--trocr-small-handwritten. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
loading configuration file config.json from cache at C:\Users\mahmoud\.cache\huggingface\hub

pytorch_model.bin:   0%|          | 0.00/246M [00:00<?, ?B/s]

loading weights file pytorch_model.bin from cache at C:\Users\mahmoud\.cache\huggingface\hub\models--microsoft--trocr-small-handwritten\snapshots\b4648cfa171985a6745f37ddd637e98c0da958ac\pytorch_model.bin
Generate config GenerationConfig {
  "eos_token_id": 2
}

Attempting to create safetensors variant
Safetensors PR exists


model.safetensors:   0%|          | 0.00/246M [00:00<?, ?B/s]

Instantiating DeiTModel model under default dtype torch.float32.
Instantiating TrOCRForCausalLM model under default dtype torch.float32.
Generate config GenerationConfig {
  "bos_token_id": 0,
  "decoder_start_token_id": 2,
  "eos_token_id": 2,
  "pad_token_id": 1,
  "use_cache": false
}

Config of the encoder: <class 'transformers.models.deit.modeling_deit.DeiTModel'> is overwritten by shared encoder config: DeiTConfig {
  "attention_probs_dropout_prob": 0.0,
  "encoder_stride": 16,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.0,
  "hidden_size": 384,
  "image_size": 384,
  "initializer_range": 0.02,
  "intermediate_size": 1536,
  "layer_norm_eps": 1e-12,
  "model_type": "deit",
  "num_attention_heads": 6,
  "num_channels": 3,
  "num_hidden_layers": 12,
  "patch_size": 16,
  "pooler_act": "tanh",
  "pooler_output_size": 384,
  "qkv_bias": true,
  "torch_dtype": "float32",
  "transformers_version": "4.51.3"
}

Config of the decoder: <class 'transformers.models.trocr.modeling_troc

generation_config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

loading configuration file generation_config.json from cache at C:\Users\mahmoud\.cache\huggingface\hub\models--microsoft--trocr-small-handwritten\snapshots\b4648cfa171985a6745f37ddd637e98c0da958ac\generation_config.json
Generate config GenerationConfig {
  "bos_token_id": 0,
  "decoder_start_token_id": 2,
  "eos_token_id": 2,
  "pad_token_id": 1,
  "use_cache": false
}



In [64]:

class BubbleSheetProcessor:
    def __init__(self, image_path):
        self.image_path = image_path
        self.nc = None
        self.image = None
        self.gray = None
        self.warped = None
        self.cropped = None
        self.thresh = None
        self.bubble_contours = None
        self.answer_matrix = None
        self.student_id=None
        self.bubbles_area_x, self.bubbles_area_y, self.bubbles_area_w, self.bubbles_area_h=None,None,None,None
        self.marker_area=None

    def load_image(self):
        self.image = cv2.imread(self.image_path)
        self.gray = cv2.cvtColor(self.image, cv2.COLOR_BGR2GRAY)

    def get_id_corner_map(self):
        aruco_dict = cv2.aruco.getPredefinedDictionary(cv2.aruco.DICT_4X4_100)
        parameters = cv2.aruco.DetectorParameters()
        detector = cv2.aruco.ArucoDetector(aruco_dict, parameters)

        corners, ids, _ = detector.detectMarkers(self.gray)
        if ids is None or len(ids) < 4:
            raise ValueError("Not all 4 ArUco markers were detected.")

        id_corner_map = {id[0]: corner for id, corner in zip(ids, corners)}

        if 48 in id_corner_map:
            self.nc = 5
        elif 49 in id_corner_map:
            self.nc = 4
        else:
            raise ValueError("Required ArUco marker not found.")
        #give the area of a circle with diameter equals to the edge of a marker
        marker_arr=id_corner_map[30]
        self.marker_area=np.pi*(int(abs(marker_arr[0][1][0]-marker_arr[0][0][0]))/2)**2
        return id_corner_map

    def get_center(self, corner):
        return corner[0].mean(axis=0)

    def get_centroid(self, array):
        return np.array(array).mean(axis=0)

    def get_roi(self, id_corner_map):
        if 48 in id_corner_map:
            ordered_ids = [30, 10, 48, 34]  # TL, TR, BR, BL
        else:  # 49 is guaranteed if 48 isn't
            ordered_ids = [30, 10, 49, 34]

        ordered_pts = [self.get_center(id_corner_map[id]) for id in ordered_ids]
        return np.array(ordered_pts, dtype='float32')

    def get_warped_image(self, src_pts):
        width, height = int(self.image.shape[0] * 0.764), self.image.shape[0]
        dst_pts = np.array([
            [0, 0],
            [width - 1, 0],
            [width - 1, height - 1],
            [0, height - 1]
        ], dtype='float32')

        M = cv2.getPerspectiveTransform(src_pts, dst_pts)
        return cv2.warpPerspective(self.image, M, (width, height))

    def get_largest_contour(self, image, drawContour=False):
        gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
        edged = cv2.Canny(gray, 30, 200)

        contours, _ = cv2.findContours(edged, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)
        largest_contour = max(contours, key=cv2.contourArea)

        if drawContour:
            cv2.drawContours(image, largest_contour, -1, (0, 255, 0), 3)
            cv2.imshow('largest contour', image)
            cv2.waitKey(0)
            cv2.destroyAllWindows()

        return largest_contour

    def warp_paper(self):
        id_corner_map = self.get_id_corner_map()
        roi = self.get_roi(id_corner_map)
        self.warped = self.get_warped_image(roi)

    def preprocess_cropped(self):
        self.bubbles_area_x, self.bubbles_area_y, self.bubbles_area_w, self.bubbles_area_h = cv2.boundingRect(self.get_largest_contour(self.warped))
        self.cropped = self.warped[self.bubbles_area_y+10:self.bubbles_area_y+self.bubbles_area_h-10, 
                                    self.bubbles_area_x+10:self.bubbles_area_x+self.bubbles_area_w-10]
        
        cropped_gray = cv2.cvtColor(self.cropped, cv2.COLOR_BGR2GRAY)
        blurred = cv2.GaussianBlur(cropped_gray, (3, 3), 0)

        self.thresh = cv2.adaptiveThreshold(
            blurred, 255, 
            cv2.ADAPTIVE_THRESH_MEAN_C, 
            cv2.THRESH_BINARY_INV, 
            25, 4
        )

    def extract_bubble_contours(self, drawContours=False):
        contours, _ = cv2.findContours(self.thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)
        bubble_contours = []

        for c in contours:
            area = cv2.contourArea(c)
            x, y, w, h = cv2.boundingRect(c)
            aspect_ratio = w / float(h)

            if area > self.marker_area and 0.9 < aspect_ratio < 1.1:
                bubble_contours.append(c)

        bubble_contours.sort(key=lambda b: self.get_centroid(b)[0][1])

        for i in range(10):  # 10 rows expected
            start = i * self.nc * 3
            end = (i + 1) * self.nc * 3
            bubble_contours[start:end] = sorted(
                bubble_contours[start:end],
                key=lambda b: self.get_centroid(b)[0][0]
            )

        if drawContours:
            output = self.cropped.copy()
            cv2.drawContours(output, bubble_contours, -1, (0, 255, 0), 2)
            output_resized = cv2.resize(output, (800, int(output.shape[0] * 800 / output.shape[1])))
            cv2.imshow('Detected Bubbles', output_resized)
            cv2.waitKey(0)
            cv2.destroyAllWindows()

        self.bubble_contours = bubble_contours

    def get_filled_bubbles_matrix(self):
        extracted = []

        for c in self.bubble_contours:
            mask = np.zeros(self.thresh.shape, dtype="uint8")
            cv2.drawContours(mask, [c], -1, 255, -1)
            masked = cv2.bitwise_and(self.thresh, self.thresh, mask=mask)

            total = cv2.countNonZero(mask)
            filled = cv2.countNonZero(masked)
            ratio = filled / total

            extracted.append(1 if ratio > 0.5 else 0)

        self.answer_matrix = np.array(extracted).reshape(-1, self.nc)
    def detect_student_id(self):
        id_area=self.warped[0:self.bubbles_area_y, int(self.warped.shape[1]*.75):]
        x, y, w, h = cv2.boundingRect(self.get_largest_contour(id_area))
        
        student_id_img=id_area[y+5:y+h-5, x+5:x+w-5]
        
        pixel_values = processor(images=student_id_img, return_tensors="pt").pixel_values
        # Run inference
        generated_ids = model.generate(pixel_values,max_length=14,min_length=4,num_beams =2,early_stopping =True)
        self.student_id = processor.batch_decode(generated_ids, skip_special_tokens=True)[0]

        
    
    def run_pipeline(self):
        self.load_image()
        self.warp_paper()
        self.preprocess_cropped()
        self.extract_bubble_contours(True)
        self.get_filled_bubbles_matrix()
        self.detect_student_id()
        return self.student_id, self.answer_matrix


In [67]:
import time
s=time.time()
sheet_processor=BubbleSheetProcessor('Downloads/pi.jpg')
print(sheet_processor.run_pipeline())
print(time.time()-s)

ValueError: cannot reshape array of size 150 into shape (4)

In [45]:
a= np.array([[[1199., 1574.],
        [1245., 1574.],
        [1244., 1620.],
        [1199., 1620.]]])

1661.9025137490005

In [ ]:
def get_results(model_answer_image, *students_answers_images):
    processor=BubbleSheetProcessor(model_answer_image)
    _,mam=processor.run_pipeline()
    
    for student_answer in students_answers_images:
        processor=BubbleSheetProcessor(student_answer)
        student_id,sam=processor.run_pipeline()
        sam==mam
        print(student_id,np.all(sam==mam,axis=1).sum())
        
    

In [ ]:
get_results('Downloads/modelanswer.jpg','Downloads/e.jpg','Downloads/pi.jpg')

In [56]:
s1=np.array([[1, 1, 0, 0, 0],
       [1, 0, 0, 0, 0],
       [1, 0, 0, 0, 0],
       [0, 1, 0, 0, 0],
       [0, 1, 0, 0, 0],
       [0, 1, 0, 0, 0],
       [0, 0, 1, 0, 0],
       [0, 0, 1, 0, 0],
       [0, 0, 1, 0, 0],
       [0, 0, 0, 1, 0],
       [0, 0, 0, 1, 0],
       [0, 0, 0, 1, 0],
       [0, 0, 0, 0, 1],
       [0, 0, 0, 0, 1],
       [0, 0, 0, 0, 1],
       [0, 0, 0, 0, 1],
       [0, 0, 0, 0, 1],
       [0, 0, 0, 0, 1],
       [0, 0, 0, 1, 0],
       [0, 0, 0, 1, 0],
       [0, 0, 0, 1, 0],
       [0, 0, 1, 0, 0],
       [0, 0, 0, 0, 0],
       [0, 0, 1, 0, 0],
       [0, 1, 0, 0, 0],
       [0, 1, 0, 0, 0],
       [0, 1, 0, 0, 0],
       [1, 0, 0, 0, 0],
       [1, 0, 0, 0, 0],
       [1, 0, 0, 0, 0]])


s2=np.array([[1, 1, 0, 0, 0],
       [0, 1, 0, 0, 0],
       [0,1, 0, 0, 0],
       [0, 1, 0, 0, 0],
       [0, 1, 0, 0, 0],
       [0, 1, 0, 0, 0],
       [0, 0, 1, 0, 0],
       [0, 0, 1, 0, 0],
       [0, 0, 1, 0, 0],
       [0, 0, 0, 1, 0],
       [0, 0, 0, 1, 0],
       [0, 0, 0, 1, 0],
       [0, 0, 0, 0, 1],
       [0, 0, 0, 0, 1],
       [0, 0, 0, 0, 1],
       [0, 0, 0, 0, 1],
       [0, 0, 0, 0, 1],
       [0, 0, 0, 0, 1],
       [0, 0, 0, 1, 0],
       [0, 0, 0, 1, 0],
       [0, 0, 0, 1, 0],
       [0, 0, 1, 0, 0],
       [0, 0, 0, 0, 0],
       [0, 0, 1, 0, 0],
       [0, 1, 0, 0, 0],
       [0, 1, 0, 0, 0],
       [0, 1, 0, 0, 0],
       [1, 0, 0, 0, 0],
       [1, 0, 0, 0, 0],
       [1, 0, 0, 0, 0]])

s3=np.array([[1, 0, 0, 0, 0],
       [1, 0, 0, 0, 0],
       [1, 0, 0, 0, 0],
       [0, 1, 0, 0, 0],
       [0, 1, 0, 0, 0],
       [0, 1, 0, 0, 0],
       [0, 0, 1, 0, 0],
       [0, 0, 0, 1, 0],
       [0, 0, 1, 0, 0],
       [0, 0, 0, 1, 0],
       [0, 0, 0, 1, 0],
       [0, 0, 1, 0, 0],
       [0, 0, 0, 0, 1],
       [0, 0, 0, 0, 1],
       [0, 0, 0, 0, 1],
       [1, 0, 0, 0, 0],
       [1, 0, 0, 0, 0],
       [1, 0, 0, 0, 0],
       [0, 1, 0, 0, 0],
       [0, 1, 0, 0, 0],
       [0, 0, 1, 0, 0],
       [0, 0, 1, 0, 0],
       [0, 0, 1, 0, 0],
       [0, 0, 1, 0, 0],
       [0, 0, 0, 1, 0],
       [0, 0, 0, 1, 0],
       [0, 0, 0, 1, 0],
       [0, 0, 0, 0, 1],
       [0, 0, 0, 0, 1],
       [0, 0, 1, 0, 0]])


In [63]:
(s1^s2).sum()

4

In [66]:
m=np.array([[1, 0, 0, 0, 0],
       [1, 0, 0, 0, 0],
       [1, 0, 0, 0, 0],
       [0, 1, 0, 0, 0],
       [0, 1, 0, 0, 0],
       [0, 1, 0, 0, 0],
       [0, 0, 1, 0, 0],
       [0, 0, 1, 0, 0],
       [0, 0, 1, 0, 0],
       [0, 0, 0, 1, 0],
       [0, 0, 0, 1, 0],
       [0, 0, 0, 1, 0],
       [0, 0, 0, 0, 1],
       [0, 0, 0, 0, 1],
       [0, 0, 0, 0, 1],
       [1, 0, 0, 0, 0],
       [1, 0, 0, 0, 0],
       [1, 0, 0, 0, 0],
       [0, 1, 0, 0, 0],
       [0, 1, 0, 0, 0],
       [0, 1, 0, 0, 0],
       [0, 0, 1, 0, 0],
       [0, 0, 1, 0, 0],
       [0, 0, 1, 0, 0],
       [0, 0, 0, 1, 0],
       [0, 0, 0, 1, 0],
       [0, 0, 0, 1, 0],
       [0, 0, 0, 0, 1],
       [0, 0, 0, 0, 1],
       [0, 0, 0, 0, 1]])